In [1]:
%matplotlib inline

import json
Settings = json.load(open('../settings.txt'))
import matplotlib.pyplot as plt
import numpy as np
from os.path import join
from cselect import color as cs
import sys
sys.path.insert(0,'../')
sys.path.insert(0,'../samples')
from mvpose.data import epfl_campus
from time import time

root = Settings['data_root']
root = join(root, 'pak')
tmp = Settings['tmp']

import mvpose.data.kth_football2 as kth
from mvpose import pose
from mvpose.settings import get_settings
from paf_loader import Loader
from mvpose.evaluation import pcp

# =====================================
params = get_settings(scale_to_mm=1000)
loader = Loader()
# =====================================

# def execute(frame):
#     global params, loader
#     _start = time()
#     Im, Y, Calib = epfl_campus.get(root, frame)
#     heatmaps, pafs = loader.load_confidence_map_and_paf(
#         'campus', Im, frame, dir=Settings['tmp'])
#     _end = time()
#     print('frame ' + str(frame) + ', elapsed:', _end - _start)

    
def generate_pcp_score(frame):
    global Settings, params
    alpha = 0.5
    
    Im, Y, Calib = epfl_campus.get(root, frame)
    heatmaps, pafs = loader.load_confidence_map_and_paf(
        'campus', Im, frame, dir=Settings['tmp'])
    detections = pose.estimate(Calib, heatmaps, pafs, 
                               settings=params, debug=False)
    
    Humans = kth.transform3d_from_mscoco(detections)
    
    L_Arms = []
    U_Arms = []
    L_Legs = []
    U_Legs = []
    GTIDs = []
  
    for gtid, gt in enumerate(Y):
        if gt is None:
            continue
        
        larms = 0
        uarms = 0
        llegs = 0
        ulegs = 0
        avg = 0
        for d in Humans:
            r = pcp.evaluate(gt, d, alpha)
            larms_ = r.lower_arms
            uarms_ = r.upper_arms
            ulegs_ = r.upper_legs
            llegs_ = r.lower_legs
            avg_ = (larms_ + uarms_ + ulegs_ + llegs_) / 4
            if avg_ > avg:
                avg = avg_
                larms = larms_
                uarms = uarms_
                llegs = llegs_
                ulegs = ulegs_
        
        L_Arms.append(larms)
        U_Arms.append(uarms)
        L_Legs.append(llegs)
        U_Legs.append(ulegs)
        GTIDs.append(gtid)
    
    del heatmaps
    del pafs
    del detections
    del Im
    del Y
    del Calib
    
    return L_Arms, U_Arms, L_Legs, U_Legs, GTIDs

# =====================================

valid_frames = list(range(350, 470)) + list(range(650, 750))
PER_GTID = {}
for frame in valid_frames:
    _start = time()
    L_Arms, U_Arms, L_Legs, U_Legs, GTIDs =\
        generate_pcp_score(frame)
    _end = time()
    print('frame ' + str(frame) + ', elapsed:', _end - _start)
    if len(L_Arms) > 0:
        for gtid, larms, uarms, llegs, ulegs in zip(
            GTIDs, L_Arms, U_Arms, L_Legs, U_Legs
        ):
            if not gtid in PER_GTID:
                PER_GTID[gtid] = {
                    'larms': [],
                    'uarms': [],
                    'llegs': [],
                    'ulegs': [],
                    'frame': []
                }
            PER_GTID[gtid]['larms'].append(larms)
            PER_GTID[gtid]['uarms'].append(uarms)
            PER_GTID[gtid]['llegs'].append(llegs)
            PER_GTID[gtid]['ulegs'].append(ulegs)
            PER_GTID[gtid]['frame'].append(frame)

/home/tanke/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


frame 350, elapsed: 1.1825456619262695
frame 351, elapsed: 1.0589807033538818
frame 352, elapsed: 1.0694024562835693
frame 353, elapsed: 1.075235366821289
frame 354, elapsed: 1.0750312805175781
frame 355, elapsed: 1.0539069175720215
frame 356, elapsed: 1.1501572132110596
frame 357, elapsed: 1.0983831882476807
frame 358, elapsed: 1.0636613368988037
frame 359, elapsed: 1.0579466819763184
frame 360, elapsed: 1.100440502166748
frame 361, elapsed: 1.086033821105957
frame 362, elapsed: 1.0759196281433105
frame 363, elapsed: 1.0698301792144775
frame 364, elapsed: 1.0573384761810303
frame 365, elapsed: 1.1060307025909424
frame 366, elapsed: 1.0942308902740479
frame 367, elapsed: 1.1209917068481445
frame 368, elapsed: 1.1207249164581299
frame 369, elapsed: 1.1574184894561768
frame 370, elapsed: 1.0645015239715576
frame 371, elapsed: 1.0913898944854736
frame 372, elapsed: 1.0634002685546875
frame 373, elapsed: 1.0581378936767578
frame 374, elapsed: 1.0620837211608887
frame 375, elapsed: 1.072809

frame 742, elapsed: 1.5182371139526367
frame 743, elapsed: 1.3802695274353027
frame 744, elapsed: 1.4966866970062256
frame 745, elapsed: 1.396134853363037
frame 746, elapsed: 2.2176198959350586
frame 747, elapsed: 1.4934666156768799
frame 748, elapsed: 2.498159170150757
frame 749, elapsed: 1.3538463115692139


In [2]:
total_avg = []
for key, values in PER_GTID.items():
    print('actor ', key)
    print('\tuarms:', np.mean(values['uarms']))
    print('\tlarms:', np.mean(values['larms']))
    print('\tulegs:', np.mean(values['ulegs']))
    print('\tllegs:', np.mean(values['llegs']))
    avg = np.mean([
        np.mean(values['uarms']),
        np.mean(values['larms']),
        np.mean(values['ulegs']),
        np.mean(values['llegs'])
    ])
    total_avg.append(avg)
    print('\tavg:  ', avg)
print('\navg*:  ', np.mean(total_avg))

actor  1
	uarms: 0.9813829787234043
	larms: 0.601063829787234
	ulegs: 1.0
	llegs: 0.9973404255319149
	avg:   0.8949468085106383
actor  2
	uarms: 0.9080882352941176
	larms: 0.6617647058823529
	ulegs: 0.9742647058823529
	llegs: 0.9742647058823529
	avg:   0.879595588235294
actor  0
	uarms: 0.6770833333333334
	larms: 0.5104166666666666
	ulegs: 0.9375
	llegs: 0.9375
	avg:   0.765625

avg*:   0.8467224655819775


## Same as Master thesis
* actor  1
    * uarms: 0.9813829787234043
    * larms: 0.601063829787234
	* ulegs: 1.0
	* llegs: 0.9973404255319149
	* avg:   0.8949468085106383
* actor  2
	* uarms: 0.9007352941176471
	* larms: 0.6544117647058824
	* ulegs: 0.9742647058823529
	* llegs: 0.9742647058823529
	* avg:   0.8759191176470588
* actor  0
	* uarms: 0.6979166666666666
	* larms: 0.5104166666666666
	* ulegs: 0.9583333333333334
	* llegs: 0.9583333333333334
	* avg:   0.78125
* avg*:   0.8507053087192323

## Same as Master thesis without Unaries
* actor  1
	* uarms: 0.9787234042553191
	* larms: 0.601063829787234
	* ulegs: 1.0
	* llegs: 0.9973404255319149
	* avg:   0.894281914893617
* actor  2
	* uarms: 0.8897058823529411
	* larms: 0.6544117647058824
	* ulegs: 0.9669117647058824
	* llegs: 0.9669117647058824
	* avg:   0.869485294117647
* actor  0
	* uarms: 0.6770833333333334
	* larms: 0.5104166666666666
	* ulegs: 0.9375
	* llegs: 0.9375
	* avg:   0.765625
* avg*:   0.843130736337088

In [3]:
# actor  1
# 	uarms: 0.9813829787234043
# 	larms: 0.598404255319149
# 	ulegs: 1.0
# 	llegs: 0.9973404255319149
# 	avg:   0.8942819148936171
# actor  2
# 	uarms: 0.9080882352941176
# 	larms: 0.6617647058823529
# 	ulegs: 0.9742647058823529
# 	llegs: 0.9742647058823529
# 	avg:   0.879595588235294
# actor  0
# 	uarms: 0.6770833333333334
# 	larms: 0.5104166666666666
# 	ulegs: 0.9375
# 	llegs: 0.9375
# 	avg:   0.765625

# avg*:   0.8465008343763037
# avg*:   0.8465008343763037
# avg*:   0.8464160930329578
# avg*:   0.8467224655819775


# actor  1
# 	uarms: 0.9813829787234043
# 	larms: 0.601063829787234
# 	ulegs: 1.0
# 	llegs: 0.9973404255319149
# 	avg:   0.8949468085106383
# actor  2
# 	uarms: 0.9044117647058824
# 	larms: 0.6580882352941176
# 	ulegs: 0.9742647058823529
# 	llegs: 0.9742647058823529
# 	avg:   0.8777573529411764
# actor  0
# 	uarms: 0.6979166666666666
# 	larms: 0.5208333333333334
# 	ulegs: 0.9583333333333334
# 	llegs: 0.9583333333333334
# 	avg:   0.7838541666666667

# avg*:   0.8521861093728272

# actor  1
# 	uarms: 0.9813829787234043
# 	larms: 0.598404255319149
# 	ulegs: 1.0
# 	llegs: 0.9973404255319149
# 	avg:   0.8942819148936171
# actor  2
# 	uarms: 0.9007352941176471
# 	larms: 0.6580882352941176
# 	ulegs: 0.9669117647058824
# 	llegs: 0.9669117647058824
# 	avg:   0.8731617647058824
# actor  0
# 	uarms: 0.6770833333333334
# 	larms: 0.5104166666666666
# 	ulegs: 0.9375
# 	llegs: 0.9375
# 	avg:   0.765625

# avg*:   0.8443562265331664